# Automatic filtering images

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
import subprocess

# Path to tesseract executable (in case it isn't in your PATH)
try:
    subprocess.call(["tesseract"])
except FileNotFoundError:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
def canny_edge_detection(image):
    edges = cv2.Canny(image, 80, 100, 2)
    plt.subplot(121), plt.imshow(image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edges, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    plt.show()
    return edges


In [ ]:
def denoising(image):
    dst = cv2.fastNlMeansDenoising(image, None)
    plt.subplot(121), plt.imshow(image, 'gray')
    plt.subplot(122), plt.imshow(dst, 'gray')
    plt.show()

    return dst

In [ ]:
def gaussian_blur(image):
    blur = cv2.GaussianBlur(image, (7, 7), 0)
    plt.subplot(121), plt.imshow(image, 'gray'), plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(blur, 'gray'), plt.title('Blurred')
    plt.xticks([]), plt.yticks([])
    plt.show()
    return blur

In [ ]:
def thresholding(image):
    ret, th3 = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

In [ ]:
def erosion(image):
    erosion_size = 5
    
    erosion_shape = cv2.MORPH_RECT

    element = cv2.getStructuringElement(erosion_shape, (2 * erosion_size + 1, 2 * erosion_size + 1),
                                        (erosion_size, erosion_size))

    erosion_dst = cv2.erode(image, element)
    plt.subplot(121), plt.imshow(image, 'gray'), plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(erosion_dst, 'gray'), plt.title('Erosion')
    plt.xticks([]), plt.yticks([])
    plt.show()
    return erosion_dst

In [ ]:
def dilation(image):
    dilatation_size = 5
    dilation_shape = cv2.MORPH_RECT
    element = cv2.getStructuringElement(dilation_shape, (2 * dilatation_size + 1, 2 * dilatation_size + 1),
                                        (dilatation_size, dilatation_size))
    dilatation_dst = cv2.dilate(image, element)
    plt.subplot(121), plt.imshow(image, 'gray'), plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(dilatation_dst, 'gray'), plt.title('Dilatation')
    plt.xticks([]), plt.yticks([])
    plt.show()
    return dilatation_dst

In [ ]:
"""if __name__ == "__main__":
    # Load the image
    img = cv2.imread('../images/003.jpg')
    
    # Create a mask of the same size as the image, initialized to 0
    mask = np.zeros(img.shape[:2], np.uint8)
    
    # Define a rectangle that includes the foreground object
    rect = (50,50,450,290)
    
    # Create two arrays, bgdModel and fgdModel, as temporary arrays for the GrabCut algorithm
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    
    # Apply the GrabCut algorithm with the appropriate parameters
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    
    # Create a mask where sure and likely foreground regions are marked with 1
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    
    # Multiply the original image with the new mask to get the result
    img = img*mask2[:,:,np.newaxis]
    
    # Display the image
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()"""

In [ ]:
if __name__ == "__main__":
    # img = cv2.imread('../images/002.png', cv2.IMREAD_GRAYSCALE)
    img = cv2.imread('../images/011.jpg',cv2.IMREAD_GRAYSCALE)
    assert img is not None, "file could not be read, check with os.path.exists()"
    #median = cv2.medianBlur(img, 21)
    median = gaussian_blur(img)

    denois = denoising(median)
    thresh = thresholding(denois)

    plt.subplot(121), plt.imshow(img, cmap='gray'), plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(thresh, 'gray'), plt.title('Thresholding')
    plt.xticks([]), plt.yticks([])
    plt.show()
    eroded = erosion(thresh)
    dilated = dilation(eroded)
    text = pytesseract.image_to_string(img, lang='ita')
    print("\033[92m{}\033[00m".format('Prima\n' + text))
    #result = cv2.bitwise_and(img, img, mask=dilated)

    text = pytesseract.image_to_string(median, lang='ita')
    print("\033[92m{}\033[00m".format('Blurring\n' + text))
    text = pytesseract.image_to_string(denois, lang='ita')
    print("\033[92m{}\033[00m".format('Denoised\n' + text))
    text = pytesseract.image_to_string(thresh, lang='ita')
    print("\033[92m{}\033[00m".format('Threshold\n' + text))
    text = pytesseract.image_to_string(eroded, lang='ita')
    print("\033[92m{}\033[00m".format('Eroded\n' + text))
    text = pytesseract.image_to_string(dilated, lang='ita')
    print("\033[92m{}\033[00m".format('Dilated\n' + text))